This jupyter notebook is to show function `babs-init`.

In [6]:
# See `README_howtorun_show_babs_init_ipynb.md` for how to run this file!

# Clone out:
#   datalad clone ria+file:///path/to/<container>/output_ria#~data singleORmulti-ses_<container>_outputs
# If you want to change `project_root`'s foldername, also change `output_ria/alias/data` symlink:
    # cd output_ria/alias
    # ln -sf /new/full/path/to/output_ria/xxx/xxx-xxx-xxx-xxx data

# flake8: noqa

# set up how Jupyter notebook behaves:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'   # print all outputs

import os
import os.path as op

There are several arguments when calling `babs-init`:

In [11]:
# This notebook only accepts toybidsapp, qsiprep or fmriprep or fmriprep_anatonly (input ds: BIDS)
# ++++++++++++++++++++++++++++++++++
flag_instance = "toybidsapp"
type_session = "multi-ses"
which_dataset = "toy_fake"    # "toy_fake", "toy_real", "HBN"

flag_where = "local"     # "cubic" or "local"
# ++++++++++++++++++++++++++++++++++

# where:
if flag_where == "cubic":
    where_root = "/cbica/projects/BABS"
    where_project = op.join(where_root, "data")
    where_notebooks = op.join(where_root, "babs/notebooks")
elif flag_where == "local":
    where_root = "/Users/cztemp/Desktop/Satterthwaite_Lab/datalad_wrapper"  # change this back after laptop's back
    where_project = op.join(where_root, "data")
    where_notebooks = op.join(where_root, "babs/notebooks")
else:
    raise Exception("not valid `flag_where`!")

# Input dataset:
if which_dataset == "toy_fake":
    if type_session == "multi-ses":
        input_ds = op.join(where_project, "w2nu3")
    elif type_session == "single-ses":
        input_ds = op.join(where_project, "t8urc")
elif which_dataset == "toy_real":  # real data:
    if type_session == "multi-ses":
        # input_ds = "/cbica/projects/RBC/chenying_practice/data_for_babs/NKI/data_hashedID_bids"
        input_ds = "/cbica/projects/BABS/data/testdata_NKI/data_hashedID_bids"
    elif type_session == "single-ses":
        raise Exception("not supported yet!")
elif which_dataset == "HBN":  # HBN data:
    if type_session == "single-ses":
        input_ds = "/cbica/projects/BABS/data/rawdata_HBN"   # datalad sibling in BABS cubic project
    elif type_session == "multi-ses":
        raise Exception("HBN is a single-ses data!")


project_name = "test_babs_" + type_session
# Based on which dataset:
if which_dataset == "HBN":   # HBN data:
    project_name += "_HBN"
# Based on which BIDS App:
if flag_instance == "toybidsapp":
    input_cli = [["BIDS", input_ds]]
    bidsapp = "toybidsapp"
    container_name = bidsapp + "-0-0-5"
elif flag_instance in ["fmriprep", "fmriprep_anatonly"]:
    bidsapp = "fmriprep"
    container_name = bidsapp + "-20-2-3"
elif flag_instance == "qsiprep":
    bidsapp = "qsiprep"
    container_name = bidsapp + "-0-16-0RC3"
else:
    raise Exception("`flag_instance` is not QSIPrep or fMRIPrep!")

project_name += "_" + flag_instance


# Container:
if flag_where == "cubic":
    container_ds = op.join(where_project, bidsapp + "-container")  
elif flag_where == "local":
    container_ds = op.join(where_project, "toybidsapp-container-docker")  # just use toybidsapp to test out
container_config_yaml_file = op.join(where_notebooks, "example_container_" + flag_instance + ".yaml")

# list_sub_file = op.join(where_notebooks, "initial_sub_list_" + type_session + ".csv")
list_sub_file = None

Let's pass these arguments into `babs-init` CLI:

In [12]:
cmd = "babs-init \\\n"
cmd += "\t" + "--where_project " + where_project + " \\\n"
cmd += "\t" + "--project_name " + project_name + " \\\n"
cmd += "\t" + "--input " + "BIDS" + " " + input_ds + " \\\n"
if list_sub_file is not None:
    cmd += "\t" + "--list_sub_file " + list_sub_file + " \\\n"
cmd += "\t" + "--container_ds " + container_ds + " \\\n"
cmd += "\t" + "--container_name " + container_name + " \\\n"
cmd += "\t" + "--container_config_yaml_file " + container_config_yaml_file + " \\\n"
cmd += "\t" + "--type_session " + type_session + " \\\n"
cmd += "\t" + "--type_system " + "sge"

print("The command to execute:")
print(cmd)


The command to execute:
babs-init \
	--where_project /Users/cztemp/Desktop/Satterthwaite_Lab/datalad_wrapper/data \
	--project_name test_babs_multi-ses_fmriprep \
	--input BIDS /Users/cztemp/Desktop/Satterthwaite_Lab/datalad_wrapper/data/w2nu3 \
	--container_ds /Users/cztemp/Desktop/Satterthwaite_Lab/datalad_wrapper/data/toybidsapp-container-docker \
	--container_name fmriprep-20-2-3 \
	--container_config_yaml_file /Users/cztemp/Desktop/Satterthwaite_Lab/datalad_wrapper/babs/notebooks/example_container_fmriprep.yaml \
	--type_session multi-ses \
	--type_system sge


In [ ]:
!($cmd)